In [11]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##### 1 task

In [107]:
def sample_size(p, rel_tol) -> int:
    n = 0
    error = rel_tol + 1
    while error > rel_tol:
        n += 1
        sigma_phi = np.sqrt((1-p)/(n*p))
        error = sigma_phi
    return n


p = 0.025
rel_tol = 0.05

n = sample_size(p, rel_tol)
print(n)

15600


##### 2 task

In [108]:
def sample_size(gamma, alpha, delta):
    error = delta + 1
    n = 1
    while error >= delta:
        z_quantile = ss.norm.ppf(1 - alpha)
        phi_z_quantile = ss.norm.pdf(z_quantile)
        error = (gamma/(6*np.sqrt(n)))*(2*z_quantile**2+1)*phi_z_quantile
        n+=1
    return n-1

result = sample_size(2, 0.025, 0.025 * 0.1)
print(result)

4579


##### 3 task

In [154]:
def pivot_interval(n, k, gamma):
    sample_mean = k/n
    z = ss.norm.ppf((1-gamma)/2)
    a = z**2 + n
    b = (2*n*sample_mean + z**2)
    c = n*sample_mean**2
    p_R = -(-b - np.sqrt(b**2-4*a*c))/(2*a)
    p_L = -(-b + np.sqrt(b**2-4*a*c))/(2*a)
    return p_L, p_R


n = 20
k = 3
gamma = 0.95

p_alpha, p_beta = pivot_interval(n, k, gamma)
print("Доверительный интервал для p:", (p_alpha, p_beta))

Доверительный интервал для p: (0.052368745896216595, 0.36041886474075696)


##### 4 task

In [155]:
def var_interval(sample, gamma):
    n = len(sample)
    alpha = 1 - gamma
    g_1 = ss.chi2.ppf(alpha / 2, n - 1)
    g_2 = ss.chi2.ppf(1 - alpha / 2, n - 1)
    sample_variance = np.var(sample, ddof=1)
    lower_bound = (n - 1) * sample_variance / g_2
    upper_bound = (n - 1) * sample_variance / g_1
    return (lower_bound, upper_bound)


print(var_interval([0, 1], 0.95))

(0.09952454760270645, 509.12913485983375)


##### 5 task

In [207]:
def loc_interval(sample, gamma, number_of_trials):
    # MONTE-CARLO for quantiles
    n = len(sample)
    loc = 1
    scale = 5
    sample_monte_carlo = ss.cauchy(loc=loc, scale=scale).rvs((number_of_trials, n))
    med = np.median(sample_monte_carlo, axis=1)
    MAD = np.median(abs(sample_monte_carlo - np.resize(med, (number_of_trials, n))), axis=1)
    g = (med - loc)/MAD
    alpha = 1 - gamma
    quantile_1 = np.quantile(g, alpha/2)
    quantile_2 = np.quantile(g, 1 - alpha/2)
    # CONFIDENCE INTERVAL
    med_loc = np.median(sample)
    MAD_loc = np.median(abs(sample - np.resize(med_loc, (1, n))))
    loc_2 = med_loc - MAD_loc*quantile_1
    loc_1 = med_loc - MAD_loc*quantile_2
    return loc_1, loc_2


sample = [-0.93, -1.84, -0.84, -0.13, -0.63, 0.06, -0.93, 13.29, 0.9, -2.64,
          -0.37, 0.43, -2.41, 19.33, -0.18, 1.29, 1.32, -0.47, -0.27, 0.27,
          1.07, -1.49, -0.78, 0.59, -0.0, -1.59, -0.28, -1.38, 0.1, 1.72]
print(loc_interval(sample, 0.95, 10000))

(-0.6050185968267746, 0.15172480236498656)


##### 6 task

In [221]:
def mixture(n, eps):
# Задаем параметры компонентов смеси
    mu1, sigma1 = 0, 1
    mu2, sigma2 = 0, 5

    # Генерируем выборку смешанной смеси
    component_choice = np.random.choice([0, 1], size=n, p=[1-eps, eps])
    sample = np.empty(n)
    for i, component in enumerate(component_choice):
        if component == 0:
            sample[i] = np.random.normal(mu1, sigma1)
        else:
            sample[i] = np.random.normal(mu2, sigma2)
    
    return sample

def ratio(eps):
    sigma_1 = 1
    sigma_2 = 5
    sigma = np.sqrt((1-eps)*sigma_1**2 + eps*sigma_2**2)
    sample = mixture(10000, eps)
    c_d_mean = sigma/np.mean(sample)
    c_d_med = sigma/np.median(sample)
    return c_d_mean/c_d_med

ratio(0)

# from scipy.optimize import root_scalar
# print(root_scalar(lambda x: ratio(x) - 1, bracket=[0, 0.5]).root)

2.3219993096382527

##### 7 task

In [19]:
def corr_interval(sample, gamma):
    # Вычислить коэффициент корреляции Пирсона
    r = np.corrcoef(sample[:, 0], sample[:, 1])[0, 1]
    n = len(sample)

    # Преобразовать коэффициент корреляции в z-переменную
    z = np.arctanh(r)

    # Вычислить стандартную ошибку оценки коэффициента корреляции
    se = 1 / np.sqrt(n - 3)

    # Найти квантили нормального распределения
    z_alpha = ss.norm.ppf(gamma / 2)
    z_beta = ss.norm.ppf(1 - gamma / 2)

    # Найти границы интервала в z-переменной
    z_lower_bound = z + z_alpha * se
    z_upper_bound = z + z_beta * se

    # Преобразовать обратно в коэффициент корреляции
    lower_bound = np.tanh(z_lower_bound)
    upper_bound = np.tanh(z_upper_bound)

    return lower_bound, upper_bound


data = np.array([(576, 3.39), (635, 3.30), (558, 2.81), (578, 3.03), (666, 3.44),
                 (580, 3.07), (555, 3.0), (661, 3.43), (651, 3.36), (605, 3.13),
                 (653, 3.12), (575, 2.74), (545, 2.76), (572, 2.88), (594, 2.96)])

print(corr_interval(data, 0.95))

(0.7690819579101165, 0.7834649109693802)


In [22]:
import numpy as np


def efron_true_proba(n, rho, gamma, number_of_trials, n_of_resamples):
    # Генерация исходной выборки с заданной корреляцией
    np.random.seed(42)
    cov_matrix = np.array([[1, rho], [rho, 1]])
    sample = np.random.multivariate_normal([0, 0], cov_matrix, size=n)

    # Бутстрапирование и оценка доверительных интервалов
    right_count = 0
    left_count = 0
    for _ in range(number_of_trials):
        # Бутстрапирование
        resamples = np.array([np.random.choice(sample, n, replace=True)
                             for _ in range(n_of_resamples)])

        # Оценка корреляций
        correlations = np.array([np.corrcoef(resample[:, 0], resample[:, 1])[
                                0, 1] for resample in resamples])

        # Вычисление границ интервалов
        lower_bound = np.percentile(correlations, (1 - gamma) * 100 / 2)
        upper_bound = np.percentile(correlations, (1 + gamma) * 100 / 2)

        # Проверка попадания истинного значения в интервал
        if rho > upper_bound:
            right_count += 1
        if rho < lower_bound:
            left_count += 1

    # Вычисление вероятностей
    proba_right = right_count / number_of_trials
    proba_left = left_count / number_of_trials

    return proba_left, proba_right


# Пример использования функции
n = 100
rho = 0.5
gamma = 0.95
number_of_trials = 1000
n_of_resamples = 100

proba_left, proba_right = efron_true_proba(
    n, rho, gamma, number_of_trials, n_of_resamples)
print("Вероятность, что rho < lower_bound:", proba_left)
print("Вероятность, что rho > upper_bound:", proba_right)

ValueError: a must be 1-dimensional

In [38]:
import numpy as np


def sample_size_mc(p, rel_tol, num_trials=1000):
    """
    Estimates the minimum sample size required using Monte Carlo method.

    Args:
    p (float): True probability of success.
    rel_tol (float): Desired relative tolerance.
    num_trials (int): Number of Monte Carlo trials.

    Returns:
    int: Estimated minimum sample size required.
    """
    sample_sizes = []
    for _ in range(num_trials):
        sample = np.random.binomial(1, p, size=1000)
        sample_mean = np.mean(sample)
        sample_std = np.sqrt(sample_mean * (1 - sample_mean) / 1000)
        sample_sizes.append(sample_std)

    sample_sizes.sort()
    min_sample_size = int(
        np.ceil(np.percentile(sample_sizes, 95) / rel_tol) ** 2)
    return min_sample_size


# Пример использования:
true_probability = 0.025
relative_tolerance = 0.05

required_sample_size = sample_size_mc(true_probability, relative_tolerance)
print("Минимальный объем выборки (Монте-Карло):", required_sample_size)

Минимальный объем выборки (Монте-Карло): 1
